In [1]:
import torch
import sympy as sp


# Scalar Valued Functions
A multivariable function of dimension n that returns a scalar value:
$$f: \mathbb{R^n}\rightarrow\mathbb{R}$$

### Example
if we evaluate $f(x,y) = x+y$ at the point (2,1), we get the scalar value of 3.

In [2]:
def f(x,y):
    return x+y

print(f(2,1))

3


# Gradient
A vector composed of partial derivative of a scalar-valued function. The gradient measures the direction and the fastest rate of increase of a function at a given point.

### Example
Given $g(x,y) = 5x^2+3xy+3y^3$,  $x=2$, and $y=3$, 

$$
\nabla g = 
    \begin{bmatrix}
        \frac{\partial f}{\partial x}\\
        \frac{\partial f}{\partial y}\\
    \end{bmatrix} = 
    \begin{bmatrix}
        10x+3y\\
        3x+9y^2\\
    \end{bmatrix}= 
    \begin{bmatrix}
        29\\
        87\\
    \end{bmatrix}
$$

In [3]:
def g(input):
    x, y = input
    return 5 * x ** 2 + 3 * x * y + 3 * y ** 3

input = torch.tensor([2.0, 3.0], requires_grad=True)
output = g(input)
output.backward()
print("Gradient:", input.grad)

Gradient: tensor([29., 87.])


# Hessian
The derivative of a the gradient of scalar-valued function

### Example
Using our previous example

$$
H = \nabla^2 g(x,y) = 
    \begin{bmatrix}
    g_{xx} & g_{xy}\\
    g_{yx} & g_{yy}
    \end{bmatrix} =
    \begin{bmatrix}
    10 & 3\\
    3 & 18y
    \end{bmatrix} = 
    \begin{bmatrix}
    10 & 3\\
    3 & 54
    \end{bmatrix}
$$

In [4]:
input = torch.tensor([2.0, 3.0], requires_grad=True)
output = g(input)
grad = torch.autograd.grad(output, input, create_graph=True)[0]

# Compute the Hessian
hessian = torch.stack([torch.autograd.grad(grad[i], input, retain_graph=True)[0] for i in range(len(input))])

print("Hessian:", hessian)

Hessian: tensor([[10.,  3.],
        [ 3., 54.]])


## Hessian Determinant

We can use the Hessian determinant $\text{det}(H)$ to find the local maxima/minima of $g$. The rules are as follows:
Given that $D(x,y) = \text{det}(H(x,y)) = g_{xx}(x,y)g_{yy}(x,y)-(g_{xy}(x,y))^2$ and critical points $(a,b)$ such that $g_x(a,b) = g_y(a,b)=0$, then
- If $D(a,b) > 0$ and $g_{xx} > 0$, then $(a,b)$ is a local minimum of $g$
- If $D(a,b) > 0$ and $g_{xx} <> 0$, then $(a,b)$ is a local maximum of $g$
- If $D(a,b) < 0$, then $(a,b)$ is a saddle point of $g$
- If $D(a,b) = 0$, then the test is inconclusive.

### Example
We first find the critical points, by solving the system of equations in the gradient. 
$$10x+3y=0 \Rightarrow y=-\frac{10}{3}x$$
$$3x+9y^2=0$$
$$\Rightarrow 3x+9(-\frac{10}{3}x)^2=3x+9(\frac{100}{9}x^2)=3x+100x^2=x(3+100x)=0$$
$$\Rightarrow x=0 \text{ and } y=0 \text{ or } x=-\frac{3}{100} \text{ and } y=(-\frac{3}{100},\frac{1}{10}) $$

Since we already know the Hessian H, we can calculate its determinant using the following formula:
$$
\text{det}
    (\begin{bmatrix}
    a & b\\
    c & d
    \end{bmatrix}) = ad-bc \Rightarrow
    (\begin{bmatrix}
    10 & 3\\
    3 & 18y
    \end{bmatrix}) = 180y-9
$$

Next, we evaluate the determinant with the critical points.
 - At $(0,0)$:
  $$\text{det}(H) = 180(0)-9=-9$$
  So, $(0,0)$ is a saddle point
 - At $(-\frac{3}{100},\frac{1}{10})$:
  $$\text{det}(H) = 180(\frac{1}{10})-9=18-9=9$$
  Since $f_xx = 10 > 0$, $(-\frac{3}{100},\frac{1}{10})$ is a local minimum




In [5]:
def f(x, y):
    return 5*x**2 + 3*x*y + 3*y**3

def hessian(f, inputs):
    return torch.autograd.functional.hessian(f, inputs)

# Find critical points using SymPy
x, y = sp.symbols('x y')
fx = 10*x + 3*y
fy = 3*x + 9*y**2
solutions = sp.solve((fx, fy), (x, y))

print("Critical points:")
for solution in solutions:
    print(solution)

# Evaluate Hessian at each critical point
for solution in solutions:
    x_crit = torch.tensor(float(solution[0]), requires_grad=True)
    y_crit = torch.tensor(float(solution[1]), requires_grad=True)
    
    hessian_at_crit = hessian(f, (x_crit, y_crit))
    hessian_tensor = torch.tensor([[hessian_at_crit[0][0], hessian_at_crit[0][1]],
                                   [hessian_at_crit[1][0], hessian_at_crit[1][1]]])
    hessian_det = torch.det(hessian_tensor)

    print(f"\nAt point {solution}:")
    print(f"Hessian determinant: {hessian_det.item()}")

    # Classify the critical point
    if hessian_det > 0:
        if hessian_tensor[0][0] > 0:
            print("This point is a local minimum")
        else:
            print("This point is a local maximum")
    elif hessian_det < 0:
        print("This point is a saddle point")
    else:
        print("The test is inconclusive")

Critical points:
(-3/100, 1/10)
(0, 0)

At point (-3/100, 1/10):
Hessian determinant: 9.0
This point is a local minimum

At point (0, 0):
Hessian determinant: -9.0
This point is a saddle point


# Vector-valued Functions
A multivariable function of dimension n that returns a vector value of dimension n:
$$h: \mathbb{R^n}\rightarrow\mathbb{R^n}$$

# The Jacobian
The Jacobian matrixis a matrix that holds all first-order partial derivatives of a vector-valued function. The Jacobian form of a vector-valued function $h(f(x,y),g(x,y))$ is :
$$
\textbf{J}h(f(x,y),g(x,y)) =
    \begin{bmatrix}
    f_x & f_y\\
    g_x & g_y
    \end{bmatrix}
$$

### Example
If 
$$
f(x,y) =
    \begin{bmatrix}
    \sin(x)+y\\
    x+\cos(y)
    \end{bmatrix}
\Rightarrow f(\pi,2\pi) =
    \begin{bmatrix}
    \sin(\pi)+2\pi\\
    \pi+\cos(2\pi)
    \end{bmatrix} =
\begin{bmatrix}
2\pi\\
\pi+1
\end{bmatrix} \approx
\begin{bmatrix}
6.28\\
4.14
\end{bmatrix}
$$
Then,
$$
\textbf{J}(f) =
    \begin{bmatrix}
    \cos(x) & 1\\
    1 & -\sin(y)
    \end{bmatrix} \Rightarrow  \textbf{J}(f(\pi,2\pi)) =
    \begin{bmatrix}
    \cos(\pi) & 1\\
    1 & -\sin(2\pi)
    \end{bmatrix} =
    \begin{bmatrix}
    -1 & 1\\
    1 & 0
    \end{bmatrix} 
$$ 

In [6]:
# Define the input variables
x = torch.tensor(torch.pi, requires_grad=True)
y = torch.tensor(2*torch.pi, requires_grad=True)

# Define the tensor function
def f(x, y):
    return torch.stack([torch.sin(x) + y, x + torch.cos(y)])

# Calculate the output
output = f(x, y)
print("Output: ", output)

# Calculate the Jacobian
jacobian = torch.round(torch.stack(torch.autograd.functional.jacobian(f, (x, y))), decimals=4)

print("Jacobian: ", jacobian)

Output:  tensor([6.2832, 4.1416], grad_fn=<StackBackward0>)
Jacobian:  tensor([[-1.,  1.],
        [ 1., -0.]])


## Jacobian Determinant

The determinant of a Jacobian matrix can tell us the amplitude in which space contracts or expands during a transformation around a point. The Jacobian determinant at a point provides a measure of how much a function locally scales areas (in 2D), volumes (in 3D), or hyper-volumes (in higher dimensions) near that point.  The sign of the Jacobian determinant indicates whether the function preserves orientation (positive determinant) or reverses it (negative determinant). If the Jacobian determinant is non-zero at a point, the function is locally invertible at that point. This means there exists a local inverse function that maps back from the range to the domain around that point. 

To do this with a 3-dimensional Jacobian matrix, we can use the framework:

$$
\text{det}(
    \begin{bmatrix}
        a_1 & a_2 & a_3\\
        b_1 & b_2 & b_3\\
        c_1 & c_2 & c_3\\
    \end{bmatrix}
) = a_1 \text{det}(
    \begin{bmatrix}
        b_2 & b_3\\
        c_2 & c_3\\
    \end{bmatrix}
) - a_2 \text{det}(
    \begin{bmatrix}
        b_1 & b_3\\
        c_1 & c_3\\
    \end{bmatrix}
) - a_3 \text{det}(
    \begin{bmatrix}
        b_1 & b_2\\
        c_1 & c_2\\
    \end{bmatrix}
)
$$


### Example
If 
$$
f(x,y,z) =
    \begin{bmatrix}
    x^2y\\
    -y+z\\
    x+z
    \end{bmatrix}
\Rightarrow f(3,1,0) =
    \begin{bmatrix}
    9\\
    -1\\
    2
    \end{bmatrix} 
$$
Then,
$$
\textbf{J}(f) =
    \begin{bmatrix}
    2xy & x^2 & 0\\
    0 & -1 & 1\\
    1 & 0 & 1\\
    \end{bmatrix} \Rightarrow  \textbf{J}(f(3,1,0)) =
    \begin{bmatrix}
    6 & 9 & 0\\
    0 & -1 & 1\\
    1 & 0 & 1\\
    \end{bmatrix}
$$ 

Then the determinant of the Jacobian matrix can be found as follows:

$$
\text{det}(
    \begin{bmatrix}
        2xy & x^2 & 0\\
        0 & -1 & 1\\
        1 & 0 & 1\\
    \end{bmatrix}
) = 2xy \text{det}(
    \begin{bmatrix}
        -1 & 1\\
        0 & 1\\
    \end{bmatrix}
) - x^2 \text{det}(
    \begin{bmatrix}
        0 & 1\\
        1 & 1\\
    \end{bmatrix}
) - 0 \text{det}(
    \begin{bmatrix}
        0 & -1\\
        1 & 0\\
    \end{bmatrix}
) \\
= 2xy(−1(1)−(1(0)))−x^2(0(1)−1(1))+0(0(0)−(−1(1)))\\
= 2xy(−1)−x^2(−1)+0\\
= -2xy+x^2
\Rightarrow
\text{det}(\textbf{J}(f(3,1,0))) = -2(3)(1)+(3)^2=3
$$

If we evaluated it at the point (1,0,2), we get the value of 1, which shows that the space does not change around this point. However if we evaluate the Jacobian determinant at (3,1,0), then we get a value of 3, indicating a tripling of local area, volume, or hyper-volume while maintaining orientation and ensuring local invertibility.

In [7]:
def f(input):
    x, y, z = input
    return torch.stack([x**2 * y, -y + z, x + z])

def jacobian_det(func, input):
    jac = torch.autograd.functional.jacobian(func, input)
    return torch.det(jac)

point = torch.tensor([3.0, 1.0, 0.0], requires_grad=True)
det = jacobian_det(f, point)

print(f"Jacobian determinant at point (3, 1, 0): {det.item()}")

Jacobian determinant at point (3, 1, 0): 2.999999761581421
